# * VINSIGHT : Data Issue

## Parameter

In [5]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Issue Transaction

In [19]:
''' Active Caller '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'TB1S000600' --Prepaid Active Caller : TMH
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    # print(f'\nCurrent Timestamp...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-04-03, 13:11:41
   -> v_start_date: 20250101

    TM_KEY_MTH  TM_KEY_DAY   METRIC_CD                  METRIC_NAME   AREA_NAME  ACTUAL_AS_OF ACTUAL_SNAP ACTUAL_AGG_MTH              PPN_TM
0       202501    20250101  TB1S000600  Prepaid Active Caller : TMH  Nationwide      20250101  14,500,948     14,500,948 2025-04-02 23:44:23
1       202501    20250102  TB1S000600  Prepaid Active Caller : TMH  Nationwide      20250102  14,566,283     14,566,283 2025-04-02 23:44:23
2       202501    20250103  TB1S000600  Prepaid Active Caller : TMH  Nationwide      20250103  14,569,051     14,569,051 2025-04-02 23:44:23
3       202501    20250104  TB1S000600  Prepaid Active Caller : TMH  Nationwide      20250104  14,561,774     14,561,774 2025-04-02 23:44:23
4       202501    20250105  TB1S000600  Prepaid Active Caller : TMH  Nationwide      20250105  14,562,484     14,562,484 2025-04-02 23:44:23
5       202501    20250106  TB1S000600  Prepaid Active Caller : T